In [0]:
import pandas as pd

In [0]:
# Define query to download data
df_load_actual = spark.sql("""
    SELECT *
    FROM curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.load_actual
""").toPandas()

In [0]:
df_load_forecast = spark.sql("""
    SELECT *
    FROM curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.load_forecast
""").toPandas()

In [0]:
df_generation = spark.sql("""
    SELECT *
    FROM curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.generation
""").toPandas()

In [0]:
df_installed_capacity = spark.sql("""
    SELECT *
    FROM curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.installed_capacity        
""").toPandas()

In [0]:
df_solar_forecast = spark.sql("""
    SELECT *
    FROM curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.solar_forecast        
""").toPandas()

In [0]:
df_wind_forecast = spark.sql("""
    SELECT *
    FROM curlybyte_solutions_rawdata_europe_grid_load.european_grid_raw.wind_forecast        
""").toPandas()

In [0]:
df_load_actual.head(10)